In [ ]:
!pip install  simpletransformers==0.63.3  > /dev/null
!git clone https://github.com/Xirider/finetune-gpt2xl.git && chmod -R 777 finetune-gpt2xl/ && cd finetune-gpt2xl && pip install -r requirements.txt

In [ ]:
def has_valid_line(input_line: str) -> bool:
    bad_words_found = []
    black_list = ["[removed]", "[deleted]"]
    for word in black_list:
        if input_line.lower().__contains__(word.lower()):
            print(f":: Line contains word {word}... Skipping")
            bad_words_found.append(input_line)

    return len(bad_words_found) == 0

In [ ]:
import csv

import pandas
import torch
from torch.utils.data import random_split

training_data_path = "/content/drive/MyDrive/RawData/daya/Yuli-Ban-training.csv"

df = pandas.read_csv(training_data_path)

conversations = list(df['TrainingString'])

valid_lines = []
for line in conversations:
    if has_valid_line(line):
        valid_lines.append(line)

generator = torch.Generator()

generator.manual_seed(0)

train_size = int(0.9 * len(conversations))

train_dataset_file, eval_dataset_file = random_split(list(valid_lines), [train_size, len(valid_lines) - train_size], generator=generator)

print(f"Train: {len(train_dataset_file)}")
print(f"Eval: {len(eval_dataset_file)}")
print(f"Total: {len(train_dataset_file)  + len(eval_dataset_file)}")

# Note: Something is still fucked up with this. It sort of works but if you look at the number of samples the trainer receives it does not report the total as printed above...
# I think to maintain parody we need to use .txt files, and it appears like the implementation takes about anything valid...
with open('/content/finetune-gpt2xl/train.csv', mode='w', encoding='utf-8') as csv_file:
    fieldnames = ['text']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for line in train_dataset_file:
        foo = repr(line)[1:-1] + "<|endoftext|>" + "\n"
        writer.writerow({'text': foo})

with open('/content/finetune-gpt2xl/validation.csv', mode='w', encoding='utf-8') as csv_file:
    fieldnames = ['text']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for line in eval_dataset_file:
        foo = repr(line)[1:-1] + "<|endoftext|>" + "\n" # Not sure if I need to scrub it like this but whatever, seems like it works...
        writer.writerow({'text': foo})

In [ ]:
!cd finetune-gpt2xl && deepspeed --num_gpus=1 run_clm.py --deepspeed ds_config.json --model_name_or_path gpt2-xl --train_file train.csv --validation_file validation.csv --do_train --do_eval --fp16 --overwrite_cache --evaluation_strategy="steps" --output_dir finetuned --eval_steps 200 --num_train_epochs 6 --gradient_accumulation_steps 2 --per_device_train_batch_size 8 --learning_rate 1e-4

In [ ]:
!cd finetune-gpt2xl && python run_generation.py --model_type=gpt2 --model_name_or_path=finetuned --length 200

In [ ]:
import re

def capture_tag(test_string: str, expected_tag: str):
    regex = r"\<\|(.*)\|\>"

    matches = re.finditer(regex, test_string, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):

        print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))

        if match.group() == expected_tag:
            return_string = test_string.replace(match.group(), "")
            return return_string

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1

            print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

text_model_generator = LanguageGenerationModel("gpt2",  f"/content/finetune-gpt2xl/finetuned", args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	}, use_cuda=True)

print("It's going to be sad day when it learns to properly spell.  I feel like this era is a fleeting moment in AI history.  We must cherish it.")
prompt = "<|soss r/dalle2|><|sot|>Detailed scientific diagram depicting the anatomy of a tomato, full colour, realistic<|sost|>https://i.imgur.com/7adBOXn.jpg<|sor u/AsterJ|>It's going to be sad day when it learns to properly spell.  I feel like this era is a fleeting moment in AI history.  We must cherish it.<|eor|><|sor|>"

import re
regex = r"\<\|(.*)\|\>"


reply = None
refresh_args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	}
while reply is None:
	for text in text_model_generator.generate(prompt=prompt, args=refresh_args, verbose=True):
		foo = text.replace(prompt, "\n")
		result = capture_tag(foo, "<|eor|>")
		if result != None:
			reply = result
			break
print(reply)

In [ ]:
!zip -r model.zip /content/finetune-gpt2xl/finetuned

In [ ]:
!cp model.zip /content/drive/MyDrive/RawData/